In [133]:
# Tiny Shakeaspeare text
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-29 11:10:33--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-04-29 11:10:35 (17.8 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [203]:
with open('input.txt','r') as f:
    text = [l.strip() for l in f.readlines()]

In [204]:
# Concat lines
text_concat = ''
for t in text:
    text_concat += t

vocab_size = len(chars)
chars = sorted(list(set(text_concat)))

print(' '.join(chars))
print(f'{vocab_size=}')

  ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
vocab_size=64


In [205]:
# Tokenizer - character level tokenizer
stoi = {c: i for i,c in enumerate(chars)}
itos = {i: c for i,c in enumerate(chars)}

encode = lambda s: [stoi[t] for t in s]
decode = lambda d: ''.join([itos[i] for i in d])

print(encode('Joao foi a praca'))
print(decode([21, 52, 38, 52, 0, 43, 52, 46, 0, 38, 0, 53, 55, 38, 40, 38]))

[21, 52, 38, 52, 0, 43, 52, 46, 0, 38, 0, 53, 55, 38, 40, 38]
Joao foi a praca


In [206]:
import torch

data = torch.tensor(encode(' '.join(text)), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115389]) torch.int64
tensor([17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9,  0, 13, 42, 43,
        52, 55, 42,  0, 60, 42,  0, 53, 55, 52, 40, 42, 42, 41,  0, 38, 51, 62,
         0, 43, 58, 55, 57, 45, 42, 55,  5,  0, 45, 42, 38, 55,  0, 50, 42,  0,
        56, 53, 42, 38, 48,  7,  0,  0, 12, 49, 49,  9,  0, 30, 53, 42, 38, 48,
         5,  0, 56, 53, 42, 38, 48,  7,  0,  0, 17, 46, 55, 56, 57,  0, 14, 46,
        57, 46, 63, 42, 51,  9,  0, 36, 52, 58,  0, 38, 55, 42,  0, 38, 49, 49,
         0, 55, 42, 56, 52, 49, 59, 42, 41,  0, 55, 38, 57, 45, 42, 55,  0, 57,
        52,  0, 41, 46, 42,  0, 57, 45, 38, 51,  0, 57, 52,  0, 43, 38, 50, 46,
        56, 45, 11,  0,  0, 12, 49, 49,  9,  0, 29, 42, 56, 52, 49, 59, 42, 41,
         7,  0, 55, 42, 56, 52, 49, 59, 42, 41,  7,  0,  0, 17, 46, 55, 56, 57,
         0, 14, 46, 57, 46, 63, 42, 51,  9,  0, 17, 46, 55, 56, 57,  5,  0, 62,
        52, 58,  0, 48, 51, 52, 60,  0, 14, 38, 46, 58, 56,  0, 24, 38, 55, 40,
      

In [207]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [208]:
block_size = 8

x = train_data[:block_size+1]
y = train_data[1:block_size+1]
# lets check the target for each sub sequence of characters
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"For {context=}, {target=}")


For context=tensor([17]), target=tensor(46)
For context=tensor([17, 46]), target=tensor(55)
For context=tensor([17, 46, 55]), target=tensor(56)
For context=tensor([17, 46, 55, 56]), target=tensor(57)
For context=tensor([17, 46, 55, 56, 57]), target=tensor(0)
For context=tensor([17, 46, 55, 56, 57,  0]), target=tensor(14)
For context=tensor([17, 46, 55, 56, 57,  0, 14]), target=tensor(46)
For context=tensor([17, 46, 55, 56, 57,  0, 14, 46]), target=tensor(57)


In [209]:
torch.manual_seed(1337)

block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print(f'{x.shape=}, {y.shape=}')

for b in range(batch_size):
    for bl in range(block_size):
        print(f"For sequence={x[b,:bl+1].tolist()}, target={y[b,bl]}")
    print(f"batch sentence: input: {decode(x[b, :].tolist())} | output: {decode(y[b, :].tolist())}")

x.shape=torch.Size([4, 8]), y.shape=torch.Size([4, 8])
For sequence=[58], target=56
For sequence=[58, 56], target=0
For sequence=[58, 56, 0], target=57
For sequence=[58, 56, 0, 57], target=55
For sequence=[58, 56, 0, 57, 55], target=38
For sequence=[58, 56, 0, 57, 55, 38], target=46
For sequence=[58, 56, 0, 57, 55, 38, 46], target=57
For sequence=[58, 56, 0, 57, 55, 38, 46, 57], target=52
batch sentence: input: us trait | output: s traito
For sequence=[29], target=9
For sequence=[29, 9], target=0
For sequence=[29, 9, 0], target=14
For sequence=[29, 9, 0, 14], target=52
For sequence=[29, 9, 0, 14, 52], target=50
For sequence=[29, 9, 0, 14, 52, 50], target=42
For sequence=[29, 9, 0, 14, 52, 50, 42], target=5
For sequence=[29, 9, 0, 14, 52, 50, 42, 5], target=0
batch sentence: input: R: Come, | output: : Come, 
For sequence=[33], target=20
For sequence=[33, 20], target=25
For sequence=[33, 20, 25], target=14
For sequence=[33, 20, 25, 14], target=16
For sequence=[33, 20, 25, 14, 16], targe

In [210]:
import math
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C) stands for batch x time x vocab_size
        B,T,C = logits.shape
        
        if targets is None:
            loss = None
        else:
            # stretching logits and targets tensors
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # equivalent to targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits,_ = self(idx) # B,T,C
            logits = logits[:, -1, :] # B,C
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(x, y)
print(logits.shape)
print(f'{loss=} - Expected Loss = {-math.log(1/65)}')

torch.Size([32, 64])
loss=tensor(4.4599, grad_fn=<NllLossBackward0>) - Expected Loss = 4.174387269895637


In [211]:
# initial index
idx = torch.zeros((1,1), dtype=torch.long) # B,T
decode(m.generate(idx, max_new_tokens=100)[0].tolist())

" TCZ3EVgkqYUlLOy$WfZsbmH'WViL-N-sy S&Z:cLAlO;NZpEy!rEOt3:S$oNGfGt;NgNIjyPlc!QUDEYar&all.YcW&tb3!EY q'"

In [217]:
# Training Bigram Model

# Defining optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size=32

# Training loop
for i in range(1000):
    x_batch, y_batch = get_batch('train')
    
    logits, loss = m(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()

print(loss.item())

2.453453779220581


In [218]:
# initial index
idx = torch.zeros((1,1), dtype=torch.long) # B,T
decode(m.generate(idx, max_new_tokens=300)[0].tolist())

' Thanendegersourave Yort cr I, tryor t, wisthore ld feephane  ple GLORENGoverspltout:cord dandwh Oxe te: gheduglll se, f hethamyo he ro ELOurad porrs p ape he weverin: omy he, at, swobyowhano UJHATOLIs I soriacowarest asewe TISoushidol d, oper ls oou freact fltit m CET: twatoo Bosend Shathyo hu k? Ri'

In [285]:
idx.shape

torch.Size([1, 1])

In [286]:
logits,_ = m(idx) # B,T, vocab_size
logits = logits[:, -1, :] # B, vocab_size
probs = F.softmax(logits, dim=-1)
idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
idx = torch.cat((idx, idx_next), dim=-1)

x_token.shape=torch.Size([1, 1, 16])
x_pos.shape=torch.Size([8, 16])
torch.Size([1, 8, 64])


In [342]:
# Adding positional embedding and linear layer to model
n_embd = 32
head_size = 16

class BigramLanguageModel_v2(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # (C, N)
        self.positional_embedding_table = nn.Embedding(block_size, n_embd) # (B, N)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        self.lm_head = nn.Linear(head_size, vocab_size) # (N, C)

        self.register_buffer('tril', torch.tril(torch.ones(T,T)))

    def forward(self, x, targets=None):
        x_token = self.token_embedding_table(x) # (B,T, n_embd) stands for batch x time x n_embd
        x_pos = self.positional_embedding_table(torch.arange(block_size)) # (T, n_embd) stands for batch x time x n_embd
        x = x_token + x_pos # (B, T, n_embd) + (T, n_embd) 
        
        k = key(x) # (B, T, head_size)
        q = query(x) # (B, T, head_size)
        
        tril = torch.tril(torch.ones(T,T))
        weights = q @ k.transpose(-2, -1) * head_size ** -0.5 # (B, T, T)
        weights = weights.masked_fill(tril == 0, float('-inf')) # This is the form of guaranteeing future time tokens do not communicate with former tokens.
        weights = F.softmax(weights, dim=-1)

        v = value(x)
        x = weights @ v # (B,T,T) @ (B,T,head_size) = (B, T, head_size)
        
        logits = self.lm_head(x) # (B, T, vocab_size)    
        
        if targets is None:
            loss = None
        else:
            # stretching logits and targets tensors
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # equivalent to targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits,_ = self(idx[:, -block_size:]) # always analysing only previous character
            logits = logits[:, -1, :] # B, vocab_size
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx

m = BigramLanguageModel_v2(vocab_size)
logits, loss = m(x, y)
print(f'{loss=} - Expected Loss = {-math.log(1/65)}')

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [310]:
# Training Bigram Model

# Defining optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size=32

# Training loop
for i in range(1000):
    x_batch, y_batch = get_batch('train')
    
    logits, loss = m(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()

print(loss.item())

2.350381851196289


In [312]:
# initial index
idx = torch.zeros((1,1), dtype=torch.long) # B,T
decode(m.generate(idx, max_new_tokens=300)[0].tolist())

" bllillumaug s OMPKE: flle AROLEZApr? Samo angrutave e t Hasif l s,-tro, hetr is the'ttouto wicketad qus lave is Moou than urotoulcis LOR: NI' s'lothonereente. Thake the'l ' bigre ch'l bels aghitur mor-n:  ': amsordece  frete t Whalld  IOMy wee bor  loo myout. ms t m trand at tw harspe nd thenoulwiar"

## Mathematical Trick in self-attention

In [169]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # Batch x Time x Channel
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [189]:
%%time
torch.manual_seed(1338)

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (T,C)
        xbow[b,t] = torch.mean(xprev, 0)

xbow

CPU times: user 2.2 ms, sys: 244 µs, total: 2.44 ms
Wall time: 2.6 ms


tensor([[[-0.2573, -1.0673],
         [ 0.2890, -1.5117],
         [-0.0741, -1.1759],
         [-0.0733, -1.0102],
         [-0.0298, -0.7626],
         [-0.0628, -0.7502],
         [-0.0277, -0.5573],
         [-0.2171, -0.5309]],

        [[ 0.6903, -0.1961],
         [ 0.3482, -0.5813],
         [ 0.3516, -0.3518],
         [ 0.5753,  0.0477],
         [ 0.2600, -0.0438],
         [ 0.1816, -0.1691],
         [ 0.1742, -0.4228],
         [ 0.1600, -0.3997]],

        [[-0.2978,  0.0172],
         [ 0.0213,  0.5928],
         [-0.2033,  0.5208],
         [-0.1053,  0.5195],
         [-0.2123,  0.0208],
         [-0.3698,  0.0656],
         [-0.2289, -0.0062],
         [-0.1179,  0.0769]],

        [[-1.7662,  0.5860],
         [-0.5895,  0.4360],
         [-0.3902,  0.2066],
         [-0.6238,  0.2022],
         [-0.6090, -0.1312],
         [-0.6004, -0.2250],
         [-0.5611, -0.0151],
         [-0.6369, -0.0644]]])

In [190]:
%%time
# We could use these matrix multiplication tricks to make the attention heads consider only past rows in their calculations
weight = torch.tril(torch.ones(T,T))
weight /= weight.sum(1, keepdims=True)

xbow2 = weight @ x # (B, T, T) @ (B, T, C)
xbow2

CPU times: user 11.3 ms, sys: 1.17 ms, total: 12.5 ms
Wall time: 2.63 ms


tensor([[[-0.2573, -1.0673],
         [ 0.2890, -1.5117],
         [-0.0741, -1.1759],
         [-0.0733, -1.0102],
         [-0.0298, -0.7626],
         [-0.0628, -0.7502],
         [-0.0277, -0.5573],
         [-0.2171, -0.5309]],

        [[ 0.6903, -0.1961],
         [ 0.3482, -0.5813],
         [ 0.3516, -0.3518],
         [ 0.5753,  0.0477],
         [ 0.2600, -0.0438],
         [ 0.1816, -0.1691],
         [ 0.1742, -0.4228],
         [ 0.1600, -0.3997]],

        [[-0.2978,  0.0172],
         [ 0.0213,  0.5928],
         [-0.2033,  0.5208],
         [-0.1053,  0.5195],
         [-0.2123,  0.0208],
         [-0.3698,  0.0656],
         [-0.2289, -0.0062],
         [-0.1179,  0.0769]],

        [[-1.7662,  0.5860],
         [-0.5895,  0.4360],
         [-0.3902,  0.2066],
         [-0.6238,  0.2022],
         [-0.6090, -0.1312],
         [-0.6004, -0.2250],
         [-0.5611, -0.0151],
         [-0.6369, -0.0644]]])

In [191]:
torch.allclose(xbow, xbow2)

True

In [192]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf')) # This is the form of guaranteeing future time tokens do not communicate with former tokens.
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

In [193]:
torch.allclose(xbow, xbow3)

True

### Self Attention Head

In [330]:
# Implementing self-attention head
n_embd    = 32
head_size = 16

x = torch.randint(high=vocab_size, size=(B,T))

# Self attention solves the problem of gathering information from the past to the current token in a data dependent way
token_embedding = nn.Embedding(vocab_size, n_embd)

key   = nn.Linear(n_embd, head_size, bias=False)
query = nn.Linear(n_embd, head_size, bias=False)
value = nn.Linear(n_embd, head_size, bias=False)

x = token_embedding(x)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
v = value(x)

tril = torch.tril(torch.ones(T,T))
weights = q @ k.transpose(-2, -1) # (B, T, T)
weights = weights.masked_fill(tril == 0, float('-inf')) # This is the form of guaranteeing future time tokens do not communicate with former tokens.
weights = F.softmax(weights, dim=-1)
x = weights @ v * head_size ** -0.5 # (B,T,T) @ (B,T,head_size) = (B, T, head_size)

x.shape

torch.Size([4, 8, 16])

In [334]:
# Weights become data dependent according to each input from the batch...
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8646, 0.1354, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4214, 0.0770, 0.5016, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3769, 0.2137, 0.1746, 0.2347, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3459, 0.2613, 0.1767, 0.1144, 0.1016, 0.0000, 0.0000, 0.0000],
         [0.2567, 0.1610, 0.0721, 0.2641, 0.0545, 0.1916, 0.0000, 0.0000],
         [0.0457, 0.4722, 0.0211, 0.0212, 0.2727, 0.0902, 0.0769, 0.0000],
         [0.0264, 0.0572, 0.0157, 0.0676, 0.0944, 0.0211, 0.0492, 0.6684]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8535, 0.1465, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1058, 0.5093, 0.3849, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8217, 0.1011, 0.0442, 0.0330, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1501, 0.2350, 0.3374, 0.1273, 0.1501, 0.0000, 0.0000, 0.0000],
         [0.0550, 0.099

In [335]:
k.var(), q.var(), weights.var()

(tensor(0.2811, grad_fn=<VarBackward0>),
 tensor(0.3214, grad_fn=<VarBackward0>),
 tensor(0.0465, grad_fn=<VarBackward0>))

In [339]:
a = torch.randn(B,T,head_size)
b = torch.randn(B,T,head_size)

w = a @ b.transpose(-2,-1) * head_size ** -0.5 # scaling w values to variance close to 1.

In [340]:
a.var(), b.var(), w.var()

(tensor(1.0302), tensor(0.9864), tensor(1.0720))